In [1]:
from pydantic import validate_arguments, ValidationError


@validate_arguments
def repeat(s: str, count: int, *, separator: bytes = b'') -> bytes:
    b = s.encode()
    return separator.join(b for _ in range(count))


a = repeat('hello', 3)
print(a)
#> b'hellohellohello'

b = repeat('x', '4', separator=' ')
print(b)
#> b'x x x x'

try:
    c = repeat('hello', 'wrong')
except ValidationError as exc:
    print(exc)
    """
    1 validation error for Repeat
    count
      value is not a valid integer (type=type_error.integer)
    """

b'hellohellohello'
b'x x x x'
1 validation error for Repeat
count
  value is not a valid integer (type=type_error.integer)


In [2]:
import os
from pathlib import Path
from typing import Pattern, Optional

from pydantic import validate_arguments, DirectoryPath


@validate_arguments
def find_file(path: DirectoryPath, regex: Pattern, max=None) -> Optional[Path]:
    for i, f in enumerate(path.glob('**/*')):
        if max and i > max:
            return
        if f.is_file() and regex.fullmatch(str(f.relative_to(path))):
            return f


# note: this_dir is a string here
this_dir = os.path.dirname(__file__)

print(find_file(this_dir, '^validation.*'))
#> /home/runner/work/pydantic/pydantic/docs/examples/validation_decorator_field_
#> alias.py
print(find_file(this_dir, '^foobar.*', max=3))
#> None

NameError: name '__file__' is not defined

In [3]:
from pydantic import validate_arguments


@validate_arguments
def pos_or_kw(a: int, b: int = 2) -> str:
    return f'a={a} b={b}'


print(pos_or_kw(1))
#> a=1 b=2
print(pos_or_kw(a=1))
#> a=1 b=2
print(pos_or_kw(1, 3))
#> a=1 b=3
print(pos_or_kw(a=1, b=3))
#> a=1 b=3


@validate_arguments
def kw_only(*, a: int, b: int = 2) -> str:
    return f'a={a} b={b}'


print(kw_only(a=1))
#> a=1 b=2
print(kw_only(a=1, b=3))
#> a=1 b=3


@validate_arguments
def pos_only(a: int, b: int = 2, /) -> str:  # python 3.8 only
    return f'a={a} b={b}'


print(pos_only(1))
#> a=1 b=2
print(pos_only(1, 2))
#> a=1 b=2


@validate_arguments
def var_args(*args: int) -> str:
    return str(args)


print(var_args(1))
#> (1,)
print(var_args(1, 2))
#> (1, 2)
print(var_args(1, 2, 3))
#> (1, 2, 3)


@validate_arguments
def var_kwargs(**kwargs: int) -> str:
    return str(kwargs)


print(var_kwargs(a=1))
#> {'a': 1}
print(var_kwargs(a=1, b=2))
#> {'a': 1, 'b': 2}


@validate_arguments
def armageddon(
    a: int,
    /,  # python 3.8 only
    b: int,
    c: int = None,
    *d: int,
    e: int,
    f: int = None,
    **g: int,
) -> str:
    return f'a={a} b={b} c={c} d={d} e={e} f={f} g={g}'


print(armageddon(1, 2, e=3))
#> a=1 b=2 c=None d=() e=3 f=None g={}
print(armageddon(1, 2, 3, 4, 5, 6, e=8, f=9, g=10, spam=11))
#> a=1 b=2 c=3 d=(4, 5, 6) e=8 f=9 g={'g': 10, 'spam': 11}

a=1 b=2
a=1 b=2
a=1 b=3
a=1 b=3
a=1 b=2
a=1 b=3
a=1 b=2
a=1 b=2
(1,)
(1, 2)
(1, 2, 3)
{'a': 1}
{'a': 1, 'b': 2}
a=1 b=2 c=None d=() e=3 f=None g={}
a=1 b=2 c=3 d=(4, 5, 6) e=8 f=9 g={'g': 10, 'spam': 11}


In [4]:
from datetime import datetime
from pydantic import validate_arguments, Field, ValidationError
from pydantic.typing import Annotated


@validate_arguments
def how_many(num: Annotated[int, Field(gt=10)]):
    return num


try:
    how_many(1)
except ValidationError as e:
    print(e)
    """
    1 validation error for HowMany
    num
      ensure this value is greater than 10 (type=value_error.number.not_gt;
    limit_value=10)
    """


@validate_arguments
def when(dt: datetime = Field(default_factory=datetime.now)):
    return dt


print(type(when()))
#> <class 'datetime.datetime'>

1 validation error for HowMany
num
  ensure this value is greater than 10 (type=value_error.number.not_gt; limit_value=10)
<class 'datetime.datetime'>


In [5]:
from pydantic import Field, validate_arguments
from pydantic.typing import Annotated


@validate_arguments
def how_many(num: Annotated[int, Field(gt=10, alias='number')]):
    return num


how_many(number=42)

42

In [6]:
from pydantic import validate_arguments, ValidationError


@validate_arguments
def slow_sum(a: int, b: int) -> int:
    print(f'Called with a={a}, b={b}')
    #> Called with a=1, b=1
    return a + b


slow_sum(1, 1)

slow_sum.validate(2, 2)

try:
    slow_sum.validate(1, 'b')
except ValidationError as exc:
    print(exc)

Called with a=1, b=1
1 validation error for SlowSum
b
  value is not a valid integer (type=type_error.integer)


In [7]:
from pydantic import validate_arguments


@validate_arguments
def repeat(s: str, count: int, *, separator: bytes = b'') -> bytes:
    b = s.encode()
    return separator.join(b for _ in range(count))


a = repeat('hello', 3)
print(a)
#> b'hellohellohello'

b = repeat.raw_function('good bye', 2, separator=b', ')
print(b)
#> b'good bye, good bye'

b'hellohellohello'
b'good bye, good bye'


In [8]:
import asyncio
from pydantic import PositiveInt, ValidationError, validate_arguments


@validate_arguments
async def get_user_email(user_id: PositiveInt):
    # `conn` is some fictional connection to a database
    email = await conn.execute('select email from users where id=$1', user_id)
    if email is None:
        raise RuntimeError('user not found')
    else:
        return email


async def main():
    email = await get_user_email(123)
    print(email)
    #> testing@example.com
    try:
        await get_user_email(-4)
    except ValidationError as exc:
        print(exc.errors())
        """
        [
            {
                'loc': ('user_id',),
                'msg': 'ensure this value is greater than 0',
                'type': 'value_error.number.not_gt',
                'ctx': {'limit_value': 0},
            },
        ]
        """


asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [9]:
from pydantic import ValidationError, validate_arguments


class Foobar:
    def __init__(self, v: str):
        self.v = v

    def __add__(self, other: 'Foobar') -> str:
        return f'{self} + {other}'

    def __str__(self) -> str:
        return f'Foobar({self.v})'


@validate_arguments(config=dict(arbitrary_types_allowed=True))
def add_foobars(a: Foobar, b: Foobar):
    return a + b


c = add_foobars(Foobar('a'), Foobar('b'))
print(c)
#> Foobar(a) + Foobar(b)

try:
    add_foobars(1, 2)
except ValidationError as e:
    print(e)
    """
    2 validation errors for AddFoobars
    a
      instance of Foobar expected (type=type_error.arbitrary_type;
    expected_arbitrary_type=Foobar)
    b
      instance of Foobar expected (type=type_error.arbitrary_type;
    expected_arbitrary_type=Foobar)
    """

Foobar(a) + Foobar(b)
2 validation errors for AddFoobars
a
  instance of Foobar expected (type=type_error.arbitrary_type; expected_arbitrary_type=Foobar)
b
  instance of Foobar expected (type=type_error.arbitrary_type; expected_arbitrary_type=Foobar)
